# Summary
In Season 12, the strike zone formula is this:
```python
if flinch:
    minroll = 0.5
else:
    minroll = 0.3
strike_max = min(0.90,
                 minroll
                 + 0.35*ruth_scaled*(1 + 0.2*pitcher_vibes)
                 + 0.1*musc_scaled
                 + 0.2*(forwardness - 0.5)
                )
if roll < strike_max:
    strike
else:
    ball
```

In Season 13, the strike zone formula is modified. The The max possible value for a strike goes DOWN from 0.90 to 0.85, and the ruthlessness coefficient goes DOWN from 0.35 to 0.30. This formula holds for Season 13 and Season 14:
```python
if flinch:
    minroll = 0.5
else:
    minroll = 0.3
strike_max = min(0.85,
                 minroll
                 + 0.3*ruth_scaled*(1 + 0.2*pitcher_vibes)
                 + 0.1*musc_scaled
                 + 0.2*(forwardness - 0.5)
                )
if roll < strike_max:
    strike
else:
    ball
```

In Season 15, the strike zone formula changes slightly *again*. The max possible value for a strike goes UP from 0.85 to 0.86, and the ruthlessness coefficient goes DOWN from 0.3 to 0.285 (a 5% decrease):
```python
if flinch:
    minroll = 0.5
else:
    minroll = 0.3
strike_max = min(0.86,
                 minroll
                 + 0.285*ruth_scaled*(1 + 0.2*pitcher_vibes)
                 + 0.1*musc_scaled
                 + 0.2*(forwardness - 0.5)
                )
if roll < strike_max:
    strike
else:
    ball
```


## Load data

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.markers import CARETLEFTBASE, CARETRIGHTBASE

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import svm
from csv_dtypes import column_dtypes

import load

In [30]:
season = 19
df = load.data("strikes", season=season)

flinch_data = False

df["ruth_vibes"] = load.player_attribute(
    # These values are mandatory: the dataframe, the player role, and the attribute
    df, 'pitcher', 'ruthlessness',
    # These are all optional, and the default values are as they appear here
    vibes=True, mods=True, items=True, broken_items=True
)
df["cold_vibes"] = load.player_attribute(df, 'pitcher', 'coldness', vibes=True, mods=True, items=True, broken_items=True)

df["ruth_scaled"] = load.player_attribute(df, 'pitcher', 'ruthlessness', vibes=False, mods=True, items=True, broken_items=True)
df["cold_scaled"] = load.player_attribute(df, 'pitcher', 'coldness', vibes=False, mods=True, items=True, broken_items=True)

df["musc_scaled"] = load.player_attribute(df, 'batter', 'musclitude', vibes=False, mods=True, items=True, broken_items=True)
df["mox_scaled"] = load.player_attribute(df, 'batter', 'moxie', vibes=False, mods=True, items=True, broken_items=True)

# center=True is the default
df["forwardness"] = load.stadium_attribute(df, 'forwardness', center=False)

In [31]:
df.columns

Index(['event_type', 'event_time', 'roll', 'passed', 'batting_team_hype',
       'pitching_team_hype', 'game_id', 'play_count', 'ball_count',
       'strike_count', 'out_count', 'home_score', 'away_score', 'inning',
       'baserunner_count', 'baserunners', 'baserunners_next', 'is_strike',
       'strike_roll', 'strike_threshold', 'fielder_roll',
       'batter_consecutive_hits', 'weather', 'season', 'day', 'runner_count',
       'top_of_inning', 'is_maximum_blaseball', 'batter_at_bats',
       'batter_file', 'batting_team_file', 'pitcher_file',
       'pitching_team_file', 'stadium_file', 'fielder_file',
       'relevant_runner_file', 'runner_on_first_file', 'runner_on_second_file',
       'runner_on_third_file', 'runner_on_third_hh_file', 'stat_relevant_data',
       'pitcher_object', 'batter_object', 'batting_team_object',
       'pitching_team_object', 'stadium_object', 'pitcher_vibes',
       'pitcher_mods', 'pitcher_name', 'batter_vibes', 'batter_mods',
       'batter_name', 'bat

In [32]:
df["ruth+cold_scaled"] = (10 * df["ruth_scaled"] + 1 * df["cold_scaled"]) / 11
df["fwd2"] = df["forwardness"] ** 2
df["fwd3"] = df["forwardness"] ** 3
df["mox-musc"] = df["mox_scaled"] - df["musc_scaled"]
df["abs_mox-musc"] = np.abs(df["mox-musc"])
df["hypediff"] = (df["pitching_team_hype"] - df["batting_team_hype"])  # * (1 + 0.2 * df["pitcher_vibes"])
df["batting_team_hybes"] = df["batting_team_hype"] * (1 + 0.2 * df["batter_vibes"])
df["pitching_team_hybes"] = df["pitching_team_hype"] * (1 + 0.2 * df["pitcher_vibes"])

df["ruth_cold_hype"] = (df["ruth+cold_scaled"] + 0.2 * df["hypediff"]) * (1 + 0.2 * df["pitcher_vibes"])

## Filter data
Handles swapping between flinch and no-flinch situations, and excluding things like ON_FIRE.

In [33]:
# df = df[df["batter_mods"].isna() & df["pitcher_mods"].isna()]
dfc = df.copy()

# dfc["pitcher_mods"] = df.apply(overunder_fix, axis=1)
# dfc = dfc[(dfc["batting_team_hype"] == 0) & (dfc["pitching_team_hype"] == 0)]
# for exclude_mod in ["ON_FIRE", "PSYCHIC", "PARASITE", "MINIMALIST", "MAXIMALIST", "CHUNKY", "SMOOTH", "LATE_TO_PARTY",
                    # "OVERPERFORMING", "UNDERPERFORMING", "HIGH_PRESSURE", "GROWTH", "SINKING_SHIP", "TRAVELING"]:
# for exclude_mod in ["ON_FIRE", "PSYCHIC", "GROWTH", "SINKING_SHIP", "TRAVELING", "LATE_TO_PARTY"]:
# for exclude_mod in ["GROWTH", "SINKING_SHIP", "TRAVELING", "LATE_TO_PARTY"]:
# for exclude_mod in ["SINKING_SHIP", "TRAVELING", "LATE_TO_PARTY"]:
# for exclude_mod in ["TRAVELING", "LATE_TO_PARTY"]:
# for exclude_mod in ["SINKING_SHIP", "LATE_TO_PARTY"]:
for exclude_mod in ["LATE_TO_PARTY", "INHABITING"]:
# for exclude_mod in ["ON_FIRE", "PARASITE", "LATE_TO_PARTY"]:
# for exclude_mod in []:
    dfc = dfc[~dfc["batter_mods"].astype(str).str.contains(exclude_mod)]
    dfc = dfc[~dfc["pitcher_mods"].astype(str).str.contains(exclude_mod)]
    dfc = dfc[~dfc["batting_team_mods"].astype(str).str.contains(exclude_mod)]
    dfc = dfc[~dfc["pitching_team_mods"].astype(str).str.contains(exclude_mod)]


for exclude_mod in ["FLINCH"]:
    dfc["flinch"] = (dfc["batter_mods"].astype(str).str.contains(exclude_mod)) & (dfc["strike_count"] == 0)
    print(dfc.groupby("flinch").size())
    if flinch_data:
        dfc = dfc[dfc["flinch"]]
    else:
        dfc = dfc[~dfc["flinch"]]
#     dfc["flinch"] = dfc["batting_team_mods"].astype(str).str.contains(exclude_mod)
# dfc.groupby("flinch").size()

# excluding one weird data point in season 18 that seems incorrect
# for exclude_points in [#("080c2816-0fff-4e55-bb9a-8dd547f4e3b1", 41),
#                       # ("7d782994-a4fb-469d-9942-e97def670e23", 168),
#                       # ("d55e620b-723c-4fb8-acca-41075a8b2823", 345),
#                       # ("95cf5ed9-4cec-44f2-8316-926c044b91e7", 378),
#                       ]:
#     dfc = dfc[~((dfc["game_id"] == exclude_points[0]) & (dfc["play_count"] == exclude_points[1]))]


flinch
False    79149
True      1992
dtype: int64


In [34]:
digest_list = [
    "event_type",
    "roll",
    "threshold",
    "batter_name",
    "pitcher_name",
    "ruth_vibes",
    "hypediff",
    "musc_scaled",
    "mox_scaled",
    "forwardness",
    "game_id",
    "play_count",
]
# dfc.groupby(["batting_team_mods"]).size()

## SVC fit

In [35]:
# X = dfc[['passed', 'roll', 'ruth_vibes', 'ballpark_forwardness', 'musc_scaled']].copy()
dfc["hypediff_vibes"] = dfc["hypediff"] * (1+0.2*dfc["pitcher_vibes"])

X = dfc[["passed",
         "roll",
          "ruth_vibes",
          "cold_vibes",
         "hypediff_vibes",
         #"ruth_cold_hype",
         "forwardness",
         # "fwd2",
         # "fwd3",
         # "musc_scaled",
         # "mox_scaled",
         "mox-musc",
         "abs_mox-musc",
         # "batting_team_hybes",
         # "pitching_team_hybes",
         #"hypediff",
         #"mox-musc_hype",
         #"abs_mox-musc_hype"
         ]].copy()
X = X[X["roll"] < 0.86]
X = X[X["forwardness"] > 0.5]
y = X["passed"]
X = X.drop(["passed"], axis=1)

pin_intercept = None
pin_intercept = 0.3
pins = [
    #(0.245, "ruth_cold_hype"),
    (0.28 * 20/23, "ruth_vibes"),
    (0.28 * 3/23, "cold_vibes"),
    (0.28 * 3/23, "hypediff_vibes"),
    (0.10, "forwardness"),
    (-0.085, "mox-musc"),
    (-0.035, "abs_mox-musc"),
    #(-0.0175, "hypediff"),
]
for val, var in pins:
    X["roll"] -= X[var] * val
    X = X.drop(var, axis=1)

sc = StandardScaler(with_mean=False)
X2 = sc.fit_transform(X)
if pin_intercept is not None:
    X2[:, 0] -= pin_intercept / sc.scale_[0]
    
print(X["roll"].max())
print(X.shape)

trainedsvm = svm.LinearSVC(dual=False, max_iter=100000, C=10000000, fit_intercept=pin_intercept is None,
                           tol=1e-10).fit(X2, y)
predictionsvm = trainedsvm.predict(X2)
print(confusion_matrix(y, predictionsvm))
print(classification_report(y, predictionsvm))
sc.inverse_transform(trainedsvm.coef_)
trainedsvm.coef_
sc.scale_
trainedsvm.score(X2, y)
print(f"score: {trainedsvm.score(X2, y)} ({len(X2) - sum(np.diag(confusion_matrix(y, predictionsvm)))} outliers)")

coef = np.true_divide(trainedsvm.coef_, sc.scale_)
coef_scaled = coef / coef[0, 0]
coef_list = coef_scaled.tolist()[0]
intercept = trainedsvm.intercept_  # - np.dot(coef, sc.mean_)
intercept_scaled = -(intercept / coef[0, 0])[0] if pin_intercept is None else pin_intercept

print(intercept_scaled, ", intercept")
for pair in pins + list(zip(-np.array(coef_list), X.columns)):
    if "roll" not in pair[1]:
        print(pair)

0.7986036784557629
(37663, 1)
[[18538     1]
 [    2 19122]]
              precision    recall  f1-score   support

       False       1.00      1.00      1.00     18539
        True       1.00      1.00      1.00     19124

    accuracy                           1.00     37663
   macro avg       1.00      1.00      1.00     37663
weighted avg       1.00      1.00      1.00     37663

score: 0.9999203462283939 (3 outliers)
0.3 , intercept
(0.24347826086956523, 'ruth_vibes')
(0.036521739130434785, 'cold_vibes')
(0.036521739130434785, 'hypediff_vibes')
(0.1, 'forwardness')
(-0.085, 'mox-musc')
(-0.035, 'abs_mox-musc')


## Calculate threshold from fit

In [36]:
if flinch_data:
    intercept = 0.50
else:
    intercept = 0.30
if season == 11:
    dfc["threshold"] = (intercept
                        + 0.35 * dfc["ruth_vibes"]
                        + 0.20 * dfc["ballpark_forwardness"]
                        + 0.10 * dfc["musc_scaled"]
                       )
    dfc["svc"] = dfc["threshold"] - intercept
    strikemax = 0.90
    dfc.loc[(dfc["threshold"] > strikemax), "threshold"] = strikemax
elif season == 12 or season == 13:
    dfc["threshold"] = (intercept
                        + 0.30 * dfc["ruth_vibes"]
                        + 0.20 * dfc["ballpark_forwardness"]
                        + 0.10 * dfc["musc_scaled"]
                       )
    dfc["svc"] = dfc["threshold"] - intercept
    strikemax = 0.85
    dfc.loc[(dfc["threshold"] > strikemax), "threshold"] = strikemax
elif season in [14, 15, 16, 17]:
    dfc["threshold"] = (intercept
                        + 0.285 * dfc["ruth_vibes"]
                        + 0.20 * dfc["ballpark_forwardness"]
                        + 0.10 * dfc["musc_scaled"]
                       )
    dfc["svc"] = dfc["threshold"] - intercept
    strikemax = 0.86
    dfc.loc[(dfc["threshold"] > strikemax), "threshold"] = strikemax
elif season == 18:
    intercept = 0.300
    dfc.loc[dfc["forwardness"] > 0.5, "threshold"] = (intercept
                        + 0.285 * dfc["ruth_cold_hype"]
                        + 0.1 * dfc["forwardness"]
                        - 0.085 * dfc["mox-musc"]
                        - 0.035 * dfc["abs_mox-musc"]
                        # + 0.0571 * dfc["hypediff"]
                       )
    dfc.loc[dfc["forwardness"] <= 0.5, "threshold"] = (intercept - 0.05
                        + 0.285 * dfc["ruth_cold_hype"]
                        + 0.2 * dfc["forwardness"]
                        - 0.085 * dfc["mox-musc"]
                        - 0.035 * dfc["abs_mox-musc"]
                        # + 0.0571 * dfc["hypediff"]
                       )
    dfc["svc"] = dfc["threshold"] - intercept
    strikemax = 0.86
    dfc.loc[(dfc["threshold"] > strikemax), "threshold"] = strikemax
elif season == 19:
    intercept = 0.300
    dfc.loc[dfc["forwardness"] > 0.5, "threshold"] = (intercept
                        + 0.28 * 20/23 * dfc["ruth_vibes"]
                        + 0.28 * 3/23* dfc["cold_vibes"]   
                        + 0.28 * 3/23 * dfc["hypediff_vibes"]
                        + 0.1 * dfc["forwardness"]
                        - 0.085 * dfc["mox-musc"]
                        - 0.035 * dfc["abs_mox-musc"]
                        #- 0.0175 * dfc["hypediff"]
                       )
    dfc.loc[dfc["forwardness"] <= 0.5, "threshold"] = (intercept - 0.05
                        + 0.28 * 20/23 * dfc["ruth_vibes"]
                        + 0.28 * 3/23* dfc["cold_vibes"]   
                        + 0.28 * 3/23 * dfc["hypediff_vibes"]
                        + 0.2 * dfc["forwardness"]
                        - 0.085 * dfc["mox-musc"]
                        - 0.035 * dfc["abs_mox-musc"]
                        #- 0.017 * dfc["hypediff"]
                       )
    dfc["svc"] = dfc["threshold"] - intercept
    strikemax = 0.86
    dfc.loc[(dfc["threshold"] > strikemax), "threshold"] = strikemax
else:
    intercept = intercept_scaled
    dfc['threshold'] = intercept_scaled
    print(intercept_scaled)
    for pair in pins + list(zip(-np.array(coef_list), X.columns)):
        if "roll" not in pair[1]:
            print(pair)
            dfc['threshold'] += pair[0] * dfc[pair[1]]
    # dfc['threshold'] += 0.2 * dfc['ballpark_forwardness']
    dfc["svc"] = dfc["threshold"] - intercept_scaled
    strikemax = 0.86
    dfc.loc[(dfc["threshold"] > strikemax), "threshold"] = strikemax

In [37]:
dfc_strikes = dfc[dfc["event_type"] == "StrikeLooking"]
dfc_flinch = dfc[dfc["event_type"] == "StrikeFlinching"]
dfc_balls = dfc[dfc["event_type"] == "Ball"]

## Find Outliers

batting_team_mods
CARCINIZATION;0;UNHOLEY;FREE_GIFT    1
FREE_GIFT                            1
POPCORN_PAYOUTS                      1
UNDERSEA;FREE_GIFT                   1
dtype: int64
pitching_team_mods
0;GOOD_RIDDANCE;A;TRAVELING         1
GOOD_RIDDANCE;A;ACIDIC;TRAVELING    1
GROWTH                              1
POPCORN_PAYOUTS                     1
dtype: int64
day
1     1
4     1
35    1
55    1
dtype: int64
season
19    4
dtype: int64


,event_type,roll,threshold,offset,hypediff,forwardness,batter_name,pitcher_name,batter_mods,pitcher_mods,batting_team_mods,pitching_team_mods,day,game_id,play_count,weather,top_of_inning
1142,Ball,0.769662,0.769772,-0.000110,1.05,0.653052,Ortiz Lopez,Coolname Galvanic,FIRE_EATER,HIGH_PRESSURE,UNDERSEA;FREE_GIFT,POPCORN_PAYOUTS,1,355e6486-5f15-4452-9608-043e85f73c98,13,7,True
63484,StrikeLooking,0.661859,0.661638,0.000221,-0.21,0.536745,Enid Marlow,Goodwin Morin III,SCATTERED;TRAVELING,GRAVITY;REPLICA;LEGENDARY,CARCINIZATION;0;UNHOLEY;FREE_GIFT,GOOD_RIDDANCE;A;ACIDIC;TRAVELING,55,57d6b2e4-16a3-4003-819c-3e9ccc782f1c,201,7,False
38122,StrikeLooking,0.823453,0.823182,0.000271,-0.39,0.505462,Jode Preston,Snyder Briggs,REPEATING,ALTERNATE;WILD,FREE_GIFT,0;GOOD_RIDDANCE;A;TRAVELING,35,d3017031-8d55-40d6-8929-d87782a7d04e,155,19,False
5185,StrikeLooking,0.617234,0.610436,0.006798,0.30,0.485645,Freemium Seraph,Cory Twelve,SWIM_BLADDER;AMBITIOUS;OVERPERFORMING,SEEKER;ALTERNATE,POPCORN_PAYOUTS,GROWTH,4,8f214df4-96e6-4ee0-8e72-2b79a20ea18b,136,7,True


In [38]:
dfc["offset"] = dfc["roll"] - dfc["threshold"]
outliers = dfc[(dfc["passed"] & (dfc["offset"] >= 0)) | (~dfc["passed"] & (dfc["offset"] <= 0))]
# outliers = outliers[outliers["forwardness"] > 0.5]

fig, ax = plt.subplots(1)
x_val = "threshold"
ax.scatter(
    outliers[~outliers["passed"]][x_val],
    outliers[~outliers["passed"]]["offset"],
    color="red",
    label="Balls",
)
ax.scatter(
    outliers[outliers["passed"]][x_val],
    outliers[outliers["passed"]]["offset"],
    color="blue",
    label="Strikes",
)
ax.set_xlabel(x_val)
ax.set_ylabel("offset")
ax.legend()

table = outliers

# print(outliers.groupby(["batter_mods"]).size().sort_values())
# print(outliers.groupby(["pitcher_mods"]).size().sort_values())
print(outliers.groupby(["batting_team_mods"]).size().sort_values())
print(outliers.groupby(["pitching_team_mods"]).size().sort_values())
print(outliers.groupby("day").size())
print(outliers.groupby("season").size())
# pd.set_option("display.max_rows", 10)  # default 60
table[#.loc[#table["pitching_team_mods"].astype(str).str.contains("FIREPROOF"),
    [
        # "home_score", "away_score", "top_of_inning", "inning",
        # "pitcher_mul",
        # "batter_mul",
        "event_type",
        "roll",
        "threshold",
        "offset",
        # "svc",
        # "ruth_vibes",
        # "cold_vibes",
        # "pitcher_ruthlessness",
        # "batter_vibes",
        # "mox_scaled",
        # "musc_scaled",
        # "ballpark_forwardness",
        # "pitcher_coldness",
        "hypediff",
        "forwardness",
        # "pitching_team_hype",
        # "batting_team_hype",
        "batter_name",
        "pitcher_name",
        "batter_mods",
        "pitcher_mods",
        "batting_team_mods",
        "pitching_team_mods",
        "day",
        "game_id",
        "play_count",
        "weather",
        #  'ball_count',
        #  'strike_count',
        #  'out_count',
        # "baserunner_count",
        # "inning",
        # "home_score",
        # "away_score",
        "top_of_inning",
    ]
].sort_values("offset")#.loc[table["pitcher_mods"].astype(str).str.contains("OVERUNDER"), :]
# pd.set_option("display.max_rows", 60)  # default 60

In [39]:
# plt.plot(np.arange(len(outlier_fraction))/len(outlier_fraction), outlier_fraction.sort_values(), '.-')
# plt.plot(np.arange(len(outlier_fraction_b))/len(outlier_fraction_b), outlier_fraction_b.sort_values(), '.-')
# pd.set_option("display.max_rows", 300)  # default 60
# print(outlier_fraction.sort_values())
# print(outliers_by_pitcher.sort_values())
# print(outlier_fraction_b.sort_values())
pd.set_option("display.max_rows", 60)  # default 60

In [40]:
for print_points in [("44a57803-045e-41aa-bc57-b76e6f50966f", 258)
                      ]:
    line = df[(df["game_id"] == print_points[0]) & (df["play_count"] == print_points[1])]
line[[
        "event_type",
        "roll",
        # "threshold",
        # "offset",
        "ruth_vibes",
        "cold_vibes",
        # "ruth_items",
        # "cold_items",
        "pitcher_vibes",
        "mox_scaled",
        "musc_scaled",
        "forwardness",
        # "pitcher_coldness",
        "hypediff",
        # "pitching_team_hype",
        # "batting_team_hype",
        "batter_name",
        "pitcher_name",
        # "batter_mods",
        # "pitcher_mods",
        # "batting_team_mods",
        # "pitching_team_mods",
        "day",
        # "game_id",
        # "play_count",
        # "weather",
        #  'ball_count',
        #  'strike_count',
        #  'out_count',
        # "baserunner_count",
        # "inning",
        # "home_score",
        # "away_score",
        # "top_of_inning",
    ]]


,event_type,roll,ruth_vibes,cold_vibes,pitcher_vibes,mox_scaled,musc_scaled,forwardness,hypediff,batter_name,pitcher_name,day


In [41]:
print(0.3
      + 0.285 * ((1.8052801115624706*10 + 0.704325)/11) +
      -0.085 * (1.135617 - 0.747848) - 0.035 * abs(1.135617 - 0.747848)
      + 0.1 * 0.563717
      + 0.057 * 0.711506
     )
print(-0.016536/(0.285*10/11))

0.8363753477230037
-0.06382315789473685


In [42]:
# pd.set_option("display.max_columns", 100)
a = line["pitcher_object"].iloc[0]
print(a.items)
print(a.ruthlessness)
print(a.data['ruthlessness'])
print(a.coldness)
print(a.data['coldness'])
# print(a.stats)
# print(1.3757098421526142 * 1.5 * (1 + 0.2 *(-0.439067)))
# print(1.2067180729529265 * 1.5 * (1 + 0.2 *(-0.439067)))
# print(1.2067180729529265 * 1.5 * (1 + 0.2 *(-0.439067)) + 0.1689917691996876 * 1 * (1 + 0.2 *(-0.439067)))

IndexError: single positional indexer is out-of-bounds

In [ ]:
b = line["batter_object"].iloc[0]
print(b.items)
print(b.moxie)
print(b.data['moxie'])
print(b.musclitude)
print(b.data['musclitude'])

## Visualize Threshold

In [ ]:
# x_val = "pitcher_ruthlessness"

plt.rcParams["font.size"] = 18
x_val = "svc"
fig, ax = plt.subplots(1, figsize=(10, 10), dpi=200, layout="constrained")

intercept_plot = intercept
# intercept_plot = -intercept_scaled
xline = np.linspace(0, 1, 201)
# xline = np.insert(xline, 1, 0.65)
yline = intercept_plot + xline
# yline = xline
yline[(yline > strikemax)] = strikemax

# ax.set_xlim(0, 1)
ax.set_ylim(0, 1)

ax.scatter(
    dfc[~dfc["passed"]][x_val],
    dfc[~dfc["passed"]]["roll"],
    color="red",
    label="Balls",
    marker=">",
)
ax.scatter(
    dfc[dfc["passed"]][x_val],
    dfc[dfc["passed"]]["roll"],
    color="blue",
    label="Strikes",
    marker="<",
)
# ax.set_xlim(-0.05, 1.05)
ax.plot(xline, yline, "k", label=f"threshold\n(max={strikemax})")
# ax.scatter(outliers[~outliers['passed']][x_val], outliers[~outliers['passed']]["roll"],
#            s=48, edgecolor='black', color="red", label="Balls (below threshold)", marker='<')
# ax.scatter(outliers[outliers['passed']][x_val], outliers[outliers['passed']]["roll"],
#            s=48, edgecolor='black', color="blue", label="Strikes (above threshold)", marker='>')
ax.set_xlabel(x_val)
ax.set_ylabel("roll")
# ax.plot(dfc["svc"].min(), 0.5, 'o')

ax.legend(loc="lower right")
fig.suptitle(f"Season {season+1}, {'' if flinch_data else 'Non-'}Flinching")
# fig.savefig(f"../figures/strikes_s{season+1}{'_flinch' if flinch_data else '_noflinch'}.png",
#             facecolor='white', dpi=200)


In [ ]:
outliers.groupby("season").size()

In [ ]:
plt.rcParams["font.size"] = 10
x_val = "svc"
fig, axes = plt.subplots(3, 2, figsize=(10, 15), constrained_layout=True)
fig.suptitle(f"Season {season+1}, {'' if flinch_data else 'Non-'}Flinching", fontsize=16)
dimlist = [5e-2, 2e-2, 1e-2, 0.5e-2, 0.2e-2, 0.1e-2]
for i, ax in enumerate(np.ravel(axes)):
    ax.scatter(
        dfc[dfc["passed"]][x_val],
        dfc[dfc["passed"]]["offset"],
        color="blue",
        label="Strike",
    )
    ax.scatter(
        dfc[~dfc["passed"]][x_val],
        dfc[~dfc["passed"]]["offset"],
        color="red",
        label="Ball",
    )
    ax.scatter(
        outliers[~outliers["passed"]][x_val],
        outliers[~outliers["passed"]]["offset"],
        color="red",
        edgecolor="black",
    )
    ax.scatter(
        outliers[outliers["passed"]][x_val],
        outliers[outliers["passed"]]["offset"],
        color="blue",
        edgecolor="black",
    )
    ax.set_xlabel(x_val)
    ax.set_ylabel("offset")
    ax.legend()
    ax.grid()
    ax.set_ylim(-dimlist[i], dimlist[i])
# fig.savefig(f"../figures/ball_strike_fit_offsets_season{season+1}{'_flinch' if flinch_data else '_noflinch'}.png",
#             facecolor='white')


In [ ]:
dfc_strikes["roll"].max()